In [ ]:
# NEW IRON (12566.768) NEW H (12818.07)

In [1]:
#importing modules
import scipy.optimize as optimize
from scipy.interpolate import griddata
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
import operator
import glob
import os
import sys
%matplotlib inline

In [ ]:
#grabbing fluxes and uncertainties from excel files
# rawobsflux = pd.read_excel("HH12_obs/Dered_Fluxes_allknots.xlsx")  # observational data read in
# rawunc = pd.read_excel("HH12_obs/Dered_Uncertanties_allknots.xlsx")

rawobsflux = pd.read_excel("HH12_obs/Dered_Fluxes.xlsx")  # observational data read in
rawunc = pd.read_excel("HH12_obs/Dered_Uncertainties.xlsx")

#grabbing values from the tabulated pandas data
obsflux = rawobsflux[-18:]
obsunc = rawunc[-18:]

obsflux = obsflux.values  # usable values: flux and uncertainty
obsunc = obsunc.values


#Deciding whether obsunc is too high...adjusting such that all uncertainties are around 1 percent of the flux at most (more realistic)
s_n_lim = 50  #setting the signal to noise ilmit

#printing to check values before 
print(obsunc)
print((obsflux/obsunc) > s_n_lim)

obsunc[(obsflux/obsunc) > s_n_lim] = 1./s_n_lim * obsflux[(obsflux/obsunc) > s_n_lim] #adjusting all uncertainties at once

print(obsunc)
print((obsflux/obsunc) > s_n_lim)

In [ ]:
# separating data by spectral line (done by hand for each line for a set of knots)
#these are defined as global variables to be used for the functions below
obsSi = obsflux[:1]  
obsFe26 = obsflux[1:2]
obsS = obsflux[2:3]
obsFe179 = obsflux[3:4]
obsNe = obsflux[4:5]
obsFe16 = obsflux[6:7]

print(obsSi)
print(obsFe26)
print(obsS)
print(obsFe179)
print(obsNe)
print(obsFe16)

In [ ]:
# raw, same for uncertainties (done by hand for each line for a set of knots)
#these are defined as global variables to be used for the functions below
runcSi = obsunc[:1]  
runcFe26 = obsunc[1:2]
runcS = obsunc[2:3]
runcFe179 = obsunc[3:4]
runcNe = obsunc[4:5]
runcFe16 = obsunc[6:7]

In [8]:
#FOR GGD 37!!!

#grabbing fluxes and uncertainties from excel files
rawobsflux = pd.read_excel("GGD37_obs/Dered_Fluxes.xlsx")  # observational data read in

#grabbing values from the tabulated pandas data
obsflux = rawobsflux[-18:]
obsflux = obsflux.values  # usable values: flux and uncertainty

#Deciding whether obsunc is too high...adjusting such that all uncertainties are around 1 percent of the flux at most (more realistic)
s_n_lim = 50.  #setting the signal to noise ilmit

#defining obsunc by the s to n value since there is no given uncertainty value from Green et al...
obsunc = obsflux / s_n_lim

# separating data by spectral line (done by hand for each line for a set of knots)
#these are defined as global variables to be used for the functions below
'''
The lines we're grabbing are 
H2s0 :1
H2s1 1:2
H2S3 2:3
H2S4 3:4
H2Sfive 4:five
H2Ssix five:six
H2S7 six:seven
H2S2 7:8

Ne128 8:9
Ne128 9:10
Fe18 10:11
Fe26 11:12

S2524 12:13
Si348 13:14
HDR 14:fifteen
PAH112 fifteen:sixteen
PAHsix2 sixteen:17

Fe5 17:18
Fe229 18:19
Fe24 19:20
Fe35 20:21
O25 21:22
Ne24 22:23
S18 23:24
S33 24:2five
Ne1 2five:2six
Ar21 2six:27
'''

obsNe128 =  obsflux[8:9]  
obsNe128 =  obsflux[9:10]  
obsFe18 =  obsflux[10:11]  
obsFe26 =  obsflux[11:12]  
obsS2524 =  obsflux[12:13]  
obsSi348 =  obsflux[13:14]  

obsFe5 =  obsflux[17:18] 
obsFe229 =  obsflux[18:19] 
obsFe24 =  obsflux[19:20]  
obsFe35 =  obsflux[20:21]  
obsO25 =  obsflux[21:22]
obsNe24 =  obsflux[22:23]
obsS18 =  obsflux[23:24]
obsS33 =  obsflux[24:25]
obsNe1 =  obsflux[25:26]
obsAr21 =  obsflux[26:27]

# raw, same for uncertainties (done by hand for each line for a set of knots)
#these are defined as global variables to be used for the functions below
runcNe128 =  obsunc[8:9]  
runcNe128 =  obsunc[9:10]  
runcFe18 =  obsunc[10:11]  
runcFe26 =  obsunc[11:12]  
runcS2524 =  obsunc[12:13]  
runcSi348 =  obsunc[13:14]  

runcFe5 =  obsunc[17:18] 
runcFe229 =  obsunc[18:19] 
runcFe24 =  obsunc[19:20]  
runcFe35 =  obsunc[20:21]  
runcO25 =  obsunc[21:22]
runcNe24 =  obsunc[22:23]
runcS18 =  obsunc[23:24]
runcS33 =  obsunc[24:25]
runcNe1 =  obsunc[25:26]
runcAr21 =  obsunc[26:27]

In [ ]:
def nonzero(full,obs):  # setting zero uncertainty values (1/3 upper limit)
    new = []
    for i in range(len(full[0])):
        if full[0][i] == 0:
            use = obs[0][i]/3.
            new.append(use)
            continue
        else:
            new.append(full[0][i])
            continue
    return(new)
            

In [ ]:
uncSi = nonzero(runcSi, obsSi)  # final uncertainties by line
uncFe26 = nonzero(runcFe26, obsFe26)
uncS = nonzero(runcS, obsS)
uncFe179 = nonzero(runcFe179, obsFe179)
uncNe = nonzero(runcNe, obsNe)
# if len(runcFe16 != 0):
uncFe16 = nonzero(runcFe16, obsFe16)

In [ ]:
#length = ["1",'2','3','4','5','6','7','8']  bad old method

#lenmid = 3311  # importing shock simulation grids
#newmid = []
#for i in range(1,lenmid+1):
#    name = "S5_Shocks/NewMid/ShockFiles/ShockSpectrum" + str(i) + ".csv"
#    data = pd.read_csv(name, names = length, engine = 'python')
#    newmid.append(data)


In [ ]:
#lenmid2 = 3311
#newmid2 = []
#for i in range(1,lenmid2+1):
#    name2 = "S5_Shocks/NewMid2/ShockFiles/ShockSpectrum" + str(i) + ".csv"
#    data2 = pd.read_csv(name2, names = length, engine = 'python')
#    newmid2.append(data2)

In [9]:
# implementing natural sorting (ref: https://stackoverflow.com/questions/5967500/how-to-correctly-sort-a-string-with-a-number-inside)

import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

In [10]:
def fileread(path): #read in the path to the shock files directory, form "S5_Shocks/NewMid/ShockFiles/ShockSpectrum*.csv"
    directory = glob.glob(path)
    directory.sort(key=natural_keys) #key=os.path.getmtime
    files = []
    length = ['1','2','3','4','5','6','7','8']
    for i in directory:
        data = pd.read_csv(i, names = length, engine = 'python')
        files.append(data)
    
    return files


In [11]:
def initial(file):  # taking the important information out of the file (wavelengths and fluxes)
    rawdata = file[-len(file)+88:]
    hbvalpoint = file[76:77]
    hbval = float(hbvalpoint.iloc[0]["4"])
    rawwl = rawdata["1"]
    wl = []
    for j in rawwl:
        num = float(j)
        u = num/10000.  # converting to microns
        wl.append(u)
    raweps = rawdata["3"]
    eps = []
    for k in raweps:
        nums = float(k)
        eps.append(nums)
    uses = []
    for a in range(len(eps)):
        vals = [eps[a]*hbval,wl[a]]
        uses.append(vals)
    return uses

In [12]:
def pointlistmake(filelist):  # making a point of (velocity,density) that maps to each wavelength intensity
    pointlist = []
    
    for file in filelist:
        fslice = file.values[5][0]
        unlist = []
        for i in (i for i,x in enumerate(fslice) if x == '_'): # finding the file points containing the velocity
            unlist.append(i)
        fvel = unlist[0] + 1

        for i in (i for i,x in enumerate(fslice) if x == 'k'):
            lvel = i
        
        vel = fslice[fvel:lvel] 

        fden = unlist[1] + 1

        for i in (i for i,x in enumerate(fslice) if x == 'g'):  # same for density
            lden = i

        den = fslice[fden:lden]

        point = [vel, den]
        pointlist.append(point)
    
    return(pointlist)

In [13]:
def modlistmake(file):
    modlist = []
    t = 0
    while t < len(file):
        lst = file[t]
        listup = initial(lst)
        
        muep = [item[0] for item in listup]
        muwl = [item[1] for item in listup]
        
        for i in (i for i,x in enumerate(muwl) if x == 34.8140927):  # 34.8 Si line
            usi = muep[i]
        
        for i in (i for i,x in enumerate(muwl) if x == 25.988404):  # 26 Fe line
            ufe1 = muep[i]
            
        for i in (i for i,x in enumerate(muwl) if x == 25.2490184):  # Sulfur line 1
            us1 = muep[i]
        
        for i in (i for i,x in enumerate(muwl) if x == 25.2):  # Sulfur line 2
            us2 = muep[i]
            
        us = us1 + us2
        
        for i in (i for i,x in enumerate(muwl) if x == 17.9360294):  # 17.9 Fe line
            ufe2 = muep[i] * (2./3.)
        
        for i in (i for i,x in enumerate(muwl) if x == 12.8135475):  # Neon line
            une = muep[i]
            
        for i in (i for i,x in enumerate(muwl) if x == 1.6435526999999999):  # 1.644 Fe line
            ufe3 = muep[i]
        
        modlist.append([usi, ufe1, us, ufe2, une, ufe3])
    
#         for i in (i for i,x in enumerate(muwl) if x == 1.6435526999999999): print i
#         for i in (i for i,x in enumerate(muwl) if x == 12.8135475): print i
#         for i in (i for i,x in enumerate(muwl) if x == 17.9360294): print i
#         for i in (i for i,x in enumerate(muwl) if x == 25.2): print i
#         for i in (i for i,x in enumerate(muwl) if x == 25.2490184): print i
#         for i in (i for i,x in enumerate(muwl) if x == 25.988404): print i
#         for i in (i for i,x in enumerate(muwl) if x == 34.8140927): print i
#         print lst
#         print listup
#         break
        
        t = t+1
        
    return(modlist)

In [14]:
def plotmodlistmake(file):  # finding the important lines for plotting
    modlist = []
    listup = initial(file)
        
    muep = [item[0] for item in listup]
    muwl = [item[1] for item in listup]
    
    for i in (i for i,x in enumerate(muwl) if x == 34.8140927):  # 34.8 Si line
        usi = muep[i]

    for i in (i for i,x in enumerate(muwl) if x == 25.988404):  # 26 Fe line
        ufe1 = muep[i]

    for i in (i for i,x in enumerate(muwl) if x == 25.2490184):  # Sulfur line 1
        us1 = muep[i]

    for i in (i for i,x in enumerate(muwl) if x == 25.2):  # Sulfur line 2
        us2 = muep[i]

    us = us1 + us2

    for i in (i for i,x in enumerate(muwl) if x == 17.9360294):  # 17.9 Fe line
        ufe2 = muep[i] * (2./3.)

    for i in (i for i,x in enumerate(muwl) if x == 12.8135475):  # Neon line
        une = muep[i]

    for i in (i for i,x in enumerate(muwl) if x == 1.6435526999999999):  # 1.644 Fe line
        ufe3 = muep[i]
    
    '''
#     for i in (i for i,x in enumerate(muwl) if x == 34.8140927):  # 34.8 Si line
    usi = muep[[i for i,x in enumerate(muwl) if x == 34.8140927][0]]
        
#     for i in (i for i,x in enumerate(muwl) if x == 25.988404):  # 26 Fe line
    ufe1 = muep[[i for i,x in enumerate(muwl) if x == 25.988404][0]]
            
#     for i in (i for i,x in enumerate(muwl) if x == 25.2490184):  # Sulfur line 1
    us1 = muep[[i for i,x in enumerate(muwl) if x == 25.2490184][0]]
        
#     for i in (i for i,x in enumerate(muwl) if x == 25.2):  # Sulfur line 2  
    us2 = muep[[i for i,x in enumerate(muwl) if x == 25.2][0]]
            
    us = us1 + us2  # for some reason it splits this sulfur line into two so we have to add
        
#     for i in (i for i,x in enumerate(muwl) if x == 17.9360294):  # 17.9 Fe line
    ufe2 = muep[[i for i,x in enumerate(muwl) if x == 17.9360294][0]] * (2./3.)
        
#     for i in (i for i,x in enumerate(muwl) if x == 12.8135475):  # Neon line
    une = muep[[i for i,x in enumerate(muwl) if x == 12.8135475][0]]
            
#     for i in (i for i,x in enumerate(muwl) if x == 1.6435526999999999):  # 1.644 Fe line
    ufe3 = muep[[i for i,x in enumerate(muwl) if x == 1.6435526999999999][0]]

    '''
    modlist.append([usi, ufe1, us, ufe2, une, ufe3])
    
        
#     1.5994774999999999, 1.610933, 1.6407209999999999, 1.6435526999999999, 1.6637705000000003, 1.6768827000000002, 1.6806538, 
#     1.7002496999999999, 1.7111315999999999, 1.7362127, 1.7449385999999998, 1.797106, 1.8000209999999999, 1.8094009999999998, 
#     1.8113858, 1.8174141, 1.8685315999999998, 1.8697216, 1.8750976, 1.9387743999999998, 1.9445582, 2.0581286, 2.1655302, 
#     2.430699, 2.4483324, 2.4693138, 2.4945739000000002, 2.5254015, 2.5636276999999996, 2.6119350999999997, 2.6251521, 2.6744016, 
#     2.7575154, 2.8722121, 3.0383731, 3.2960924, 3.5731327, 3.6059873, 3.644932, 3.6916295, 3.7395355, 3.7483739999999997, 
#     3.8184127999999995, 3.9064862000000002, 4.020870599999999, 4.0522694, 4.0763185, 4.0819575, 4.1149943, 4.1707971, 4.376458,
#     4.4348336999999995, 4.6076622, 4.6537737, 4.671941, 4.672512, 4.8891388, 5.0623453000000005, 5.128661699999999, 
#     5.3401686999999995, 5.673907, 5.9082197999999995, 6.6359642999999995, 6.985274499999999, 7.4598413, 7.5, 7.5025019, 
#     10.6821629, 10.7, 11.3, 11.3333548, 12.371909, 12.728800199999998, 12.8135475, 14.367816099999999, 14.4, 17.9360294, 
#     22.9023718, 24.0, 24.5191789, 25.2, 25.2490184, 25.988404, 33.2811928, 33.4, 34.2, 34.8140927, 35.3486794, 
#     35.777279300000004, 
    
    return(modlist)

In [15]:
def initialize(path): #"Documents/GGD37/Odd_Vel/ShockSpectrum*.csv", all you should need to do is run this function
    files = fileread(path)
    #felist = femodlistmake(files)
    modlist = modlistmake(files)
    pointlist = pointlistmake(files)
    #ratiolist = fe_ratios(felist)
    
    return files, pointlist, modlist

In [16]:
def matchfind(nplist, scalelist, idx, pointlist):  # finding the scale value, parameters of matched shock
    ipos = nplist[idx]
    scale = scalelist[idx]
    print(ipos)
    pos = pointlist[ipos]
    need = [ipos, pos, scale]
    return(need)
    

In [17]:
def chisqAll(scale, obslist, unclist, modlist):  # finding chisquare value for all lines
    slist = []
    alpha, beta, delta, epsilon, zeta = obslist[0], obslist[1], obslist[2], obslist[3], obslist[4]
    uncalpha, uncbeta, uncdelta, uncepsilon, unczeta = unclist[0], unclist[1], unclist[2], unclist[3], unclist[4]
    
    w = 0
    while w < len(modlist):  # basic chisquared minimization
        chisq = ((alpha - (modlist[w][0])*scale)/
        uncalpha)**2 + ((beta - (modlist[w][1])*scale)/
        uncbeta)**2  + ((delta - (modlist[w][3])*scale)/
        uncdelta)**2 + ((epsilon - (modlist[w][4])*scale)/
        uncepsilon)**2 + ((zeta - (modlist[w][5])*scale)/unczeta)**2
        
        slist.append(chisq)
        w = w + 1
    val = min(slist)
    for i in (i for i,x in enumerate(slist) if x == val):  
        idx = i
    ret = [val, idx]
    return(ret)


In [18]:
def allmatch(pos, modlist, pointlist): # getting all information about match, not including S
    slist = []
    nplist = []
    scalelist = []
    idxlist = []
    
    alpha = obsSi[0][pos]
    beta = obsFe26[0][pos]
    delta = obsFe179[0][pos]
    epsilon = obsNe[0][pos]
    zeta = obsFe16[0][pos]
    
    uncalpha = uncSi[pos]
    uncbeta = uncFe26[pos]
    uncdelta = uncFe179[pos]
    uncepsilon = uncNe[pos]
    unczeta = uncFe16[pos]
    
    obslist = [alpha, beta, delta, epsilon, zeta]
    unclist = [uncalpha, uncbeta, uncdelta, uncepsilon, unczeta]
    
    scale = 10**(-17)
    minim = 1500.
    while scale < 10**(-10):  # setting cutoff minimum chi squared value
        nminit = chisqAll(scale, obslist, unclist, modlist)
        nmin = nminit[0]
        idxlist.append(nminit[1])
        if nmin < minim:
            minim = nmin
            slist.append(nmin)
            nplist.append(nminit[1])
        else:
            slist.append(float("inf"))
            nplist.append(float("inf"))
        
        scalelist.append(scale)
        scale = scale*1.005
    val = min(slist)
    for i in (i for i,x in enumerate(slist) if x == min(slist)):  
        idx = i
    
    values = matchfind(nplist, scalelist, idx, pointlist)
    return(val,values)
        

In [19]:
def chisqAllred(scale, obslist, unclist, modlist):  # chisquare for just 4 important lines (Si, Ne, Fe26, Fe179)
    slist = []
    alpha, beta, delta, epsilon = obslist[0], obslist[1], obslist[2], obslist[3]
    uncalpha, uncbeta, uncdelta, uncepsilon = unclist[0], unclist[1], unclist[2], unclist[3]
    
    w = 0
    while w < len(modlist):
        chisq = ((alpha - (modlist[w][0])*scale)/
        uncalpha)**2 + ((beta - (modlist[w][1])*scale)/
        uncbeta)**2 + ((delta - (modlist[w][3])*scale)/
        uncdelta)**2 + ((epsilon - (modlist[w][4])*scale)/
        uncepsilon)**2
        
        slist.append(chisq)
        w = w + 1
    val = min(slist)
    idx = np.argmin(slist)
    ret = [val, idx]
    return(ret)

In [20]:
def allmatchred(pos, modlist, pointlist):  # match for just 4 important lines 
    slist = []
    nplist = []
    scalelist = []
    idxlist = []
    
    alpha = obsSi[0][pos]  # important lines: Fe 26, 17.9, Si, Ne
    beta = obsFe26[0][pos]
    delta = obsFe179[0][pos]
    epsilon = obsNe[0][pos]
    
    uncalpha = uncSi[pos]
    uncbeta = uncFe26[pos]
    uncdelta = uncFe179[pos]
    uncepsilon = uncNe[pos]
    
    obslist = [alpha, beta, delta, epsilon]
    unclist = [uncalpha, uncbeta, uncdelta, uncepsilon]
    
    scale = 10**(-17)
    minim = 1500.
    while scale < 10**(-10):
        nminit = chisqAllred(scale, obslist, unclist, modlist)
        nmin = nminit[0]
        idxlist.append(nminit[1])
        
        if nmin <= minim:
            minim = nmin
            slist.append(nmin)
            nplist.append(nminit[1])
        else:
            slist.append(float("inf"))
            nplist.append(float("inf"))
        
        scalelist.append(scale)
        scale = scale*1.005
    
    val = min(slist)
    idx = np.argmin(slist)
    values = matchfind(nplist, scalelist, idx, pointlist)
    return(val,values)

In [21]:
def chisq_g(scale, obslist, unclist, modlist):  # finding chisquare value for all lines
    slist = []
    alpha, beta, delta, zeta = obslist[0], obslist[1], obslist[3], obslist[5]
    uncalpha, uncbeta, uncdelta, unczeta = unclist[0], unclist[1], unclist[3], unclist[5]
    
    w = 0
    while w < len(modlist):  # basic chisquared minimization
        chisq = ((alpha - (modlist[w][0])*scale)/
        uncalpha)**2 + ((beta - (modlist[w][1])*scale)/
        uncbeta)**2  + ((delta - (modlist[w][3])*scale)/
        uncdelta)**2 + ((zeta - (modlist[w][5])*scale)/unczeta)**2
        
        slist.append(chisq)
        w = w + 1
    val = min(slist)
    for i in (i for i,x in enumerate(slist) if x == val):  
        idx = i
    ret = [val, idx]
    return(ret)

In [22]:
def allmatch_g(pos, modlist, pointlist): # getting all information about match
    slist = []
    nplist = []
    scalelist = []
    idxlist = []
    
    alpha = obsSi[0][pos]
    beta = obsFe26[0][pos]
    #gamma = obsS[0][pos]
    delta = obsFe179[0][pos]
#     epsilon = obsNe[0][pos]
    zeta = obsFe16[0][pos]
    
    uncalpha = uncSi[pos]
    uncbeta = uncFe26[pos]
    #uncgamma = uncS[pos]
    uncdelta = uncFe179[pos]
#     uncepsilon = uncNe[pos]
    unczeta = uncFe16[pos]
    
    obslist = [alpha, beta, '''gamma''', delta, '''epsilon''', zeta]
    unclist = [uncalpha, uncbeta, '''uncgamma''', uncdelta, '''uncepsilon''', unczeta]
    
    scale = 10**(-17)
    minim = 1500.
    while scale < 10**(-10):
        nminit = chisq_g(scale, obslist, unclist, modlist)
        nmin = nminit[0]
        idxlist.append(nminit[1])
        if nmin < minim:
            minim = nmin
            slist.append(nmin)
            nplist.append(nminit[1])
        else:
            slist.append(float("inf"))
            nplist.append(float("inf"))
        
        scalelist.append(scale)
        scale = scale*1.005
    val = min(slist)
    for i in (i for i,x in enumerate(slist) if x == min(slist)):  
        idx = i
    
    values = matchfind(nplist, scalelist, idx, pointlist)
    return(val,values)
        

In [23]:
def chisq_Fe_only(scale, obslist, unclist, modlist):  # finding chisquare value for all lines
    slist = []
    beta, delta, zeta = obslist[1], obslist[2], obslist[4]
    uncbeta, uncdelta, unczeta = unclist[1], unclist[2], unclist[4]
    
    w = 0
    while w < len(modlist):  # basic chisquared minimization
        chisq = ((beta - (modlist[w][1])*scale)/
        uncbeta)**2  + ((delta - (modlist[w][3])*scale)/
        uncdelta)**2 + ((zeta - (modlist[w][5])*scale)/unczeta)**2
        
        slist.append(chisq)
        w = w + 1
    val = min(slist)
    for i in (i for i,x in enumerate(slist) if x == val):  
        idx = i
    ret = [val, idx]
    return(ret)

In [24]:
def allmatch_Fe_only(pos, modlist, pointlist): # trying to only match to Fe lines
    slist = []
    nplist = []
    scalelist = []
    idxlist = []
    
#     alpha = obsSi[0][pos]
    beta = obsFe26[0][pos]
    delta = obsFe179[0][pos]
#     epsilon = obsNe[0][pos]
    zeta = obsFe16[0][pos]
    
#     uncalpha = uncSi[pos]
    uncbeta = uncFe26[pos]
    uncdelta = uncFe179[pos]
#     uncepsilon = uncNe[pos]
    unczeta = uncFe16[pos]
    
    obslist = ['''alpha''', beta, delta, '''epsilon''', zeta]
    unclist = ['''uncalpha''', uncbeta, uncdelta, '''uncepsilon''', unczeta]
    
    scale = 10**(-18)
    minim = 1500.
    while scale < 10**(-10):  # setting cutoff minimum chi squared value
        nminit = chisq_Fe_only(scale, obslist, unclist, modlist)
        nmin = nminit[0]
        idxlist.append(nminit[1])
        if nmin < minim:
            minim = nmin
            slist.append(nmin)
            nplist.append(nminit[1])
        else:
            slist.append(float("inf"))
            nplist.append(float("inf"))
        
        scalelist.append(scale)
        scale = scale*1.005
    val = min(slist)
    for i in (i for i,x in enumerate(slist) if x == min(slist)):  
        idx = i
    
    values = matchfind(nplist, scalelist, idx, pointlist)
    
    return(val,values)
        

In [25]:
def chisq_si_ne_fe16(scale, obslist, unclist, modlist):  # finding chisquare value for all lines
    slist = []
    alpha, epsilon, zeta = obslist[0], obslist[4], obslist[5]
    uncalpha, uncepsilon, unczeta = unclist[0], unclist[4], unclist[5]
    
    w = 0
    while w < len(modlist):  # basic chisquared minimization
        chisq = ((alpha - (modlist[w][0])*scale)/
        uncalpha)**2 + ((epsilon - (modlist[w][4])*scale)/
        uncepsilon)**2 + ((zeta - (modlist[w][5])*scale)/unczeta)**2
        
        slist.append(chisq)
        w = w + 1
    val = min(slist)
    for i in (i for i,x in enumerate(slist) if x == val):  
        idx = i
    ret = [val, idx]
    return(ret)


In [26]:
def allmatch_si_ne_fe16(pos, modlist, pointlist): # getting all information about match
    slist = []
    nplist = []
    scalelist = []
    idxlist = []
    
    alpha = obsSi[0][pos]
    #beta = obsFe26[0][pos]
    #gamma = obsS[0][pos]
    #delta = obsFe179[0][pos]
    epsilon = obsNe[0][pos]
    zeta = obsFe16[0][pos]
    
    uncalpha = uncSi[pos]
    #uncbeta = uncFe26[pos]
    #uncgamma = uncS[pos]
    #uncdelta = uncFe179[pos]
    uncepsilon = uncNe[pos]
    unczeta = uncFe16[pos]
    
    obslist = [alpha, '''beta''', '''gamma''', '''delta''', epsilon, zeta]
    unclist = [uncalpha, '''uncbeta''', '''uncgamma''', '''uncdelta''', uncepsilon, unczeta]
    
    scale = 10**(-17)
    minim = 1500
    while scale < 10**(-10):
        nminit = chisq_si_ne_fe16(scale, obslist, unclist, modlist)
        nmin = nminit[0]
        idxlist.append(nminit[1])
        if nmin < minim:
            minim = nmin
            slist.append(nmin)
            nplist.append(nminit[1])
        else:
            slist.append(float("inf"))
            nplist.append(float("inf"))
        
        scalelist.append(scale)
        scale = scale*1.005
    val = min(slist)
    for i in (i for i,x in enumerate(slist) if x == min(slist)):  
        idx = i
    
    values = matchfind(nplist, scalelist, idx, pointlist)
    return(val,values)

In [27]:
def chisq_twolines(scale, obslist, unclist, modlist):  # finding chisquare value for all lines
    slist = []
    beta, zeta = obslist[1], obslist[5]
    uncbeta, unczeta = unclist[1], unclist[5]
    
    
    sys.exit('Exit')
    
    w = 0
    while w < len(modlist):  # basic chisquared minimization
        chisq = ((beta - (modlist[w][1])*scale)/uncbeta)**2 + ((zeta - (modlist[w][5])*scale)/unczeta)**2
        
        slist.append(chisq)
        w = w + 1
    val = min(slist)
    for i in (i for i,x in enumerate(slist) if x == val):  
        idx = i
    ret = [val, idx]
    return(ret)


In [28]:
def allmatch_twolines(pos, modlist, pointlist): # getting all information about match
    slist = []
    nplist = []
    scalelist = []
    idxlist = []
    
#     alpha = obsSi[0][pos]
    beta = obsFe26[0][pos]
    #gamma = obsS[0][pos]
    #delta = obsFe179[0][pos]
#     epsilon = obsNe[0][pos]
    zeta = obsFe16[0][pos]
    
#     uncalpha = uncSi[pos]
    uncbeta = uncFe26[pos]
    #uncgamma = uncS[pos]
    #uncdelta = uncFe179[pos]
#     uncepsilon = uncNe[pos]
    unczeta = uncFe16[pos]
    
    obslist = ['''alpha''', beta, '''gamma''', '''delta''', '''epsilon''', zeta]
    unclist = ['''uncalpha''', uncbeta, '''uncgamma''', '''uncdelta''', '''uncepsilon''', unczeta]
    
    scale = 10**(-17)
    minim = 2e10
    while scale < 10**(-10):
        nminit = chisq_twolines(scale, obslist, unclist, modlist)
        nmin = nminit[0]
        idxlist.append(nminit[1])
        if nmin < minim:
            minim = nmin
            slist.append(nmin)
            nplist.append(nminit[1])
        else:
            slist.append(float("inf"))
            nplist.append(float("inf"))
        
        scalelist.append(scale)
        scale = scale*1.005
    val = min(slist)
    for i in (i for i,x in enumerate(slist) if x == min(slist)):  
        idx = i
    
    values = matchfind(nplist, scalelist, idx, pointlist)
    return(val,values)

In [29]:
def plotspectra(file, pos, offset): # creating a wavelength vs. intensity plot for each file, with observational data
    plot = initial(file)
    ep = [item[0] for item in plot]
    wl = [item[1] for item in plot]
    
    special = plotmodlistmake(file) # [si, fe26, s, fe179, ne, fe16]
    sep = special[0]
    swl = [34.8140927, 25.988404, 25.2490184, 17.9360294, 12.8135475, 1.6435527]
    
    feplot1 = (obsFe16[0][pos])/offset
    feplot2 = (obsFe179[0][pos])/offset
    feplot3 = (obsFe26[0][pos])/offset
    siplot = (obsSi[0][pos])/offset
    neplot = (obsNe[0][pos])/offset
    splot = (obsS[0][pos])/offset
    
    ufeplot1 = (uncFe16[pos])/offset
    ufeplot2 = (uncFe179[pos])/offset
    ufeplot3 = (uncFe26[pos])/offset
    usiplot = (uncSi[pos])/offset
    uneplot = (uncNe[pos])/offset
    usplot = (uncS[pos])/offset
    
    '''
    plotfe1 = [1.6435527, feplot1]
    plotfe2 = [17.9360294, feplot2]
    plotfe3 = [25.988404, feplot3]
    plotsi = [34.8140927, siplot]
    plotne = [12.8135475, neplot]
    plots = [25.2490184, splot]
    '''
    
    obswl = [1.6435527, 17.9360294, 25.988404, 34.8140927, 12.8135475, 25.2490184]
    obsep = [feplot1, feplot2, feplot3, siplot, neplot, splot]
    interest = [siplot, feplot3, splot, feplot2, neplot, feplot1]
    #plotting = [plotfe1, plotfe2, plotfe3, plotsi, plotne, plots]
    
    fig, ax = plt.subplots(figsize = (14,8))
    
    #ax.errorbar(obswl,obsep, yerr = obsun, fmt='o', capsize=5)
    
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_ylabel('Flux (HB = 1)')
    ax.set_xlabel(r'Wavelength ($\rm \mu m$)')
    ax.annotate("Fe 1.64", xy=(1.6435527+.1, feplot1))
    ax.annotate("Fe 17.9", xy=(17.9360294-2.2, feplot2+.025))
    ax.annotate("Fe 26", xy=(25.988404-.7, feplot3+.025))
    ax.annotate("Si", xy=(34.8140927+1.5, siplot))
    ax.annotate("Ne", xy=(12.8135475-1, neplot+.015))
    ax.annotate("S", xy=(25.2490184+1.2, splot))
    ax.stem(obswl, obsep, 'b', markerfmt='bo')
    ax.stem(wl, ep, 'g', markerfmt='go')
    ax.stem(swl, sep, 'r', markerfmt='ro')
    plt.savefig("hh7large.jpg")
    plt.show()
    
    print("Observed:", interest)
    print("Simulated:", sep)
    print("Differences: Si" ,interest[0]-sep[0], ", Fe26", interest[1]-sep[1], ", S", interest[2]-sep[2],
          ", Fe179", interest[3]-sep[3], ", Ne", interest[4]-sep[4], ", Fe16", interest[5]-sep[5])
    

In [30]:
def plotspectrasmall(file, pos, offset, figname): # creating a wavelength vs. intensity plot for each file, with observational data
    plot = initial(file)
    ep = [item[0] for item in plot]
    wl = [item[1] for item in plot]
    
    special = plotmodlistmake(file) # [si, fe26, s, fe179, ne, fe16]
    sep = special[0]
    swl = [34.8140927, 25.988404, 25.2490184, 17.9360294, 12.8135475, 1.6435527]
            
    feplot1 = (obsFe16[0][pos])/offset
    feplot2 = (obsFe179[0][pos])/offset
    feplot3 = (obsFe26[0][pos])/offset
    siplot = (obsSi[0][pos])/offset
    neplot = (obsNe[0][pos])/offset
    splot = (obsS[0][pos])/offset
    
    ufeplot1 = (uncFe16[pos])/offset
    ufeplot2 = (uncFe179[pos])/offset
    ufeplot3 = (uncFe26[pos])/offset
    usiplot = (uncSi[pos])/offset
    uneplot = (uncNe[pos])/offset
    usplot = (uncS[pos])/offset
    
    '''
    plotfe1 = [1.6435527, feplot1]
    plotfe2 = [17.9360294, feplot2]
    plotfe3 = [25.988404, feplot3]
    plotsi = [34.8140927, siplot]
    plotne = [12.8135475, neplot]
    plots = [25.2490184, splot]
    '''
    
    obswl = [1.6435527, 17.9360294, 25.988404, 34.8140927, 12.8135475, 25.2490184]
    obsep = [feplot1, feplot2, feplot3, siplot, neplot, splot]
    interest = [siplot, feplot3, splot, feplot2, neplot, feplot1]
    #plotting = [plotfe1, plotfe2, plotfe3, plotsi, plotne, plots]
        
    rwl = wl[-120:]
    rep = ep[-120:]
    
    obsun = [ufeplot1, ufeplot2, ufeplot3, usiplot, uneplot, usplot]
    
    
    fig, ax = plt.subplots(figsize = (14,8))
    
    ax.errorbar(obswl,obsep, yerr = obsun, fmt='o', capsize=14)
    
    ax.set_xlim(1,65)
    ax.set_yscale('log')
    ax.set_xscale('log')
    ax.set_ylabel('Flux (erg/sec/cm^2)')
    ax.set_xlabel(r'Wavelength ($\rm \mu m$)')
    ax.annotate("Fe 1.64", xy=(1.6435527+.1, feplot1))
    ax.annotate("Fe 17.9", xy=(17.9360294+1.1, feplot2))
    ax.annotate("Fe 26", xy=(25.988404+1.2, feplot3))
    ax.annotate("Si", xy=(34.8140927+1.5, siplot))
    ax.annotate("Ne", xy=(12.8135475-.25, neplot))
    ax.annotate("S", xy=(25.2490184+1.2, splot))
    ax.stem(obswl, obsep, 'b', markerfmt='bo')
    ax.stem(rwl, rep, 'g', markerfmt='go')
    ax.stem(swl, sep, 'r', markerfmt='ro')
    plt.savefig(figname+".jpg")
    plt.show()
    
    print("Observed:", interest)
    print("Simulated:", sep)
    print("Differences: Si" ,interest[0]-sep[0], ", Fe26", interest[1]-sep[1], ", S", interest[2]-sep[2],
          ", Fe179", interest[3]-sep[3], ", Ne", interest[4]-sep[4], ", Fe16", interest[5]-sep[5])
    

In [ ]:
shock_paths = ["S5_Shocks/NewMid/ShockFiles/ShockSpectrum*.csv", 
               "fdauphin/7_23_Spec_BGrid/BMF/ShockFiles/ShockSpectrum*.txt",
     'fdauphin/7_31_Grid_2/ShockFiles/ShockSpectrum*.txt', 
               'fdauphin/8_02_B_Grid/ShockFiles/ShockSpectrum*.txt',
 'fdauphin/8_02_B_grid_200/ShockFiles/ShockSpectrum*.txt',
  'fdauphin/7_13_B_Low/ShockFiles/ShockSpectrum*.txt',
   'fdauphin/7_16_B_low/ShockFiles/ShockSpectrum*.txt',
    'fdauphin/7_16_B_high/ShockFiles/ShockSpectrum*.txt',
   'fdauphin/7_17_B_low/ShockFiles/ShockSpectrum*.txt',
   'fdauphin/7_19_B_low/ShockFiles/ShockSpectrum*.txt',
   'fdauphin/7_19_B_high/ShockFiles/ShockSpectrum*.txt',  
    'fdauphin/7_18_no_B_high/ShockFiles/ShockSpectrum*.txt',
    'fdauphin/7_18_no_B_low/ShockFiles/ShockSpectrum*.txt',
    'fdauphin/June_2018/ShockFiles/ShockSpectrum*.txt',
    'arubinst_mods/Fe_abund/ShockSpectrum*.csv']

print(len(shock_paths))

In [ ]:
#Trying a specific case
#just need to set ind, knot_num

ind = 0
knot_num = 0

newmid, pointnewmid, modnewmid = initialize(shock_paths[ind])

match_output = allmatchred(knot_num, modnewmid, pointnewmid)
plotspectrasmall(newmid[match_output[1][0]],knot_num, match_output[1][2], 'Matching_Cases/'+rawobsflux.columns[knot_num]+'_path'+str(ind))

In [ ]:
#looping through specific paths and knots, so you can try multiple at once without trying everything
#Changed both loops to be for loops and now testing specific paths from the shock_path list and specific knot_nums

for i in [-1]:
    ind = i

    newmid, pointnewmid, modnewmid = initialize(shock_paths[ind])
    
    for j in [0,1,2,3,4,5,6]:
        knot_num = j

        try:
            #The inputs are knot number, the list of models, and the list of parameters?
            match_output = allmatch_twolines(knot_num, modnewmid, pointnewmid)
            print('Success:', i, shock_paths[i], 'Knot:', j)
            print(match_output)
            #The outputs are the chi square, the best matching spectrum / its index, the model params for the best match, a scaling factor
        
            plotspectrasmall(newmid[match_output[1][0]],knot_num, match_output[1][2], rawobsflux.columns[knot_num]+'_path'+str(i))
        except Exception as e: print(i, shock_paths[i], 'Knot:', j, e) #outputs the path number, the knot number upon failing
        
        
        j += 1

In [ ]:
'''
Note:
allmatch works for all Fe lines, Ne, Si (everything except S)
allmatchred works for all those lines except [FeII] at 1.6 microns
allmatch_g excludes the [NeII] line
allmatch_noFe excludes all three Fe lines and only fits Ne and Si
allmatch_Fe_only tries to fit all three Fe lines (17.9, 26, and 1.6)

One other thing you can try is adjusing the fourth input I put for plotspectrasmall. You should try putting it 
into a specific folder.
'''
#looping through paths to test all different shock spectra models
#The outer loop is set by the len of shock_paths, so it shouldn't need to be changed if you setup your list of paths

#variables used twice, just lengths used for loops
shock_len = len(shock_paths)
knots_len = len(rawobsflux.columns[2:])+1

chisq_list = np.zeros((shock_len,knots_len+1)) #This is a list to store chi squared values, which can be used to search through best fits

for i in [13]: #range(shock_len):
    newmid, pointnewmid, modnewmid = initialize(shock_paths[i])
   
    #looping through knots, j starts at 2 since that's the column where the observations start
    #Here you shouldn't need to change anything either. The idea is to have j <= the total number of knots you have
    j = 0
    while j <= knots_len:
        knot_num = j  
        
#         print allmatch_si_ne_fe16(knot_num, modnewmid, pointnewmid)
        
        #attempting a match for a given knot number and path
        try:
            #The inputs are knot number, the list of models, and the list of parameters?
            #The outputs are the chi square, the best matching spectrum / its index, the model params for the best match, a scaling factor
            match_output = allmatch_Fe_only(knot_num, modnewmid, pointnewmid)
            print('Success:', i, shock_paths[i], 'Knot:', j)
            print(match_output)
            
            #updating list of chi squares to later check for matches
            chisq_list[i, j] = match_output[0]
            
            #plots when model matches observations
            plotspectrasmall(newmid[match_output[1][0]],knot_num, match_output[1][2], 'Matching_Cases/'+rawobsflux.columns[knot_num]+'_path'+str(i))
        
        except Exception as e: print(i, shock_paths[i], 'Knot:', j, e) #outputs the path number, the knot number upon failing
        
        j += 1

In [ ]:
'''
Determining best matches by min chi square if more than one grid works
'''
print(chisq_list)
# print chisq_list[0][5]

#Looping through each knot
for k in [0,1,2,3,4,5,6]:
    test_list = [] #resetting list of chi square vals
    
    #looping through each path
    for l in range(12):
        test_list.append(chisq_list[l][k])  #creating a list of chi squares for a single knot
    
    m = min(i for i in test_list if i > 0)  #grabbing all values greater than 0 and finding the min value
    print("Position:", test_list.index(m))
    print("Chi Square Value:", m)
#     min_value = np.argmin(x) #another way to find the chi squared value if necessary
    
    print('Path:', shock_paths[test_list.index(m)])

In [ ]:
'''
The following tests Natalie's observations of HH 7...11
'''

rawobsflux = pd.read_excel("nallen_obs/Dered_Fluxes.xlsx")  # observational data read in
obsflux = rawobsflux[-18:]
rawunc = pd.read_excel("nallen_obs/Dered_Uncertanties.xlsx")
obsunc = rawunc[-18:]

obsflux = obsflux.values  # usable values: flux and uncertainty
obsunc = obsunc.values


obsSi = obsflux[:1]  # separating data by spectral line
obsFe26 = obsflux[1:2]
obsS = obsflux[2:3]
obsFe179 = obsflux[3:4]
obsNe = obsflux[4:5]
obsFe16 = obsflux[13:14]


runcSi = obsunc[:1]  # raw, same for uncertainties
runcFe26 = obsunc[1:2]
runcS = obsunc[2:3]
runcFe179 = obsunc[3:4]
runcNe = obsunc[4:5]
runcFe16 = obsunc[13:14]


uncSi = nonzero(runcSi, obsSi)  # final uncertainties by line
uncFe26 = nonzero(runcFe26, obsFe26)
uncS = nonzero(runcS, obsS)
uncFe179 = nonzero(runcFe179, obsFe179)
uncNe = nonzero(runcNe, obsNe)
uncFe16 = nonzero(runcFe16, obsFe16)

print(rawobsflux)
print(obsflux)

In [ ]:
'''
Attempting to fit the observations
'''
#looping through paths to test all different shock spectra models
#The outer loop is set by the len of shock_paths, so it shouldn't need to be changed if you setup your list of paths

chisq_list = np.zeros((11,7)) #This is a list to store chi squared values, which can be used to search through best fits

for i in [0]:
    ind = i

    newmid, pointnewmid, modnewmid = initialize(shock_paths[ind])
   
    #looping through knots, j starts at 2 since that's the column where the observations start
    #Here you shouldn't need to change anything either. The idea is to have j <= the total number of knots you have
    j = 0
    while j <= 6:
        knot_num = j  
                
        #attempting a match for a given knot number and path
        try:
            #The inputs are knot number, the list of models, and the list of parameters?
            #The outputs are the chi square, the best matching spectrum / its index, the model params for the best match, a scaling factor
            match_output = allmatch_Fe_only(knot_num, modnewmid, pointnewmid)
            print 'Success:', ind, shock_paths[ind], 'Knot:', j
            print match_output
            
            #updating list of chi squares to later check for matches
            chisq_list[i, j-2] = match_output[0]
            
            #plots when model matches observations
            plotspectrasmall(newmid[match_output[1][0]],knot_num, match_output[1][2], 'Match_Figures/'+rawobsflux.columns[knot_num]+'_path'+str(ind))
            
        except: 
            print 'Fail:', ind, shock_paths[ind], 'Knot:', j #outputs the path number, the knot number upon failing
            
        j += 1
        print j

In [ ]:
def hydrogen(file):
    modlist = []
    t = 0
    while t < len(file):
        lst = file[t]
        listup = initial(lst)

        muep = [item[0] for item in listup]
        muwl = [item[1] for item in listup]
        
        for i in (i for i,x in enumerate(muwl) if x == 0.6562819000000001):  # H alpha
            ua = muep[i]

        for i in (i for i,x in enumerate(muwl) if x == 1.281807):  # Paschen Beta
            ub = muep[i]
        
        ratio = ua/ub
        modlist.append(ratio)
        t = t + 1
    return(modlist)
            

In [ ]:
test = hydrogen(newmid)

In [ ]:
more = hydrogen(mega)

In [ ]:
print(more)